In [1]:
import zmq
import datetime
import pandas as pd
import numpy as np
import numpy
from numpy import inf

import json
import plotly_stream as plyst
import plotly.tools as plyt
import plotly.plotly as ply
#!pip install plotly
import tpqib
import datetime

#from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import matplotlib.pyplot as plt

import pickle

iterations = 0
df = pd.DataFrame()
pdf= pd.DataFrame()
final=pd.DataFrame()

context = zmq.Context()

#Forwarding ML output
socket_pub = context.socket(zmq.PUB)
socket_pub.bind('tcp://127.0.0.1:7040')

## from Dropbox/trading_system/system_L2P0/preprocess.ipynb
port = "7010"
# socket to talk to server
socket_sub = context.socket(zmq.SUB)
print ("Collecting & plotting stock prices.")
socket_sub.connect("tcp://localhost:%s" % port)

socket_sub.setsockopt_string(zmq.SUBSCRIBE, u'SPY')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import csv
import glob

In [17]:
# here processed data is from 7040
df_ml=pd.DataFrame()

def normalise(df,window_length=60):
    #df=data[['askPrice','askSize','bidPrice','bidSize','mid','vwap','spread','v','ret','sigma']]
    #dfn=(df-df.rolling(window_length).min())/(df.rolling(window_length).max()-df.rolling(window_length).min())
    #dfn['sym']=data.sym
    #dfn.insert(0,'sym',data.sym)
    mn=df[['askPrice','askSize','bidPrice','bidSize','mid','vwap','spread','v','ret','sigma']].rolling(60).min()
    mx=df[['askPrice','askSize','bidPrice','bidSize','mid','vwap','spread','v','ret','sigma']].rolling(60).max()
    dfn=df[['askPrice','askSize','bidPrice','bidSize','mid','vwap','spread','v','ret','sigma']].rolling(1).mean()
    diff1=(dfn.dropna()-mn.dropna())
    diff2=(mx.dropna()-mn.dropna())
    diff=diff1/diff2
    diff.insert(0,'sym',df.Stock)
    return diff

def de_normalise(data,df,window_length=60):
    dn=(df*(data.rolling(window_length).max()-data.rolling(window_length).min()))+data.rolling(window_length).min()
    return dn

#https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

##### ARIMA        

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima_model import ARIMAResults
        
###ARIMA preprocessing
def arima_processing(df):
    #data=df[['vwap','mid']]
    df=df[['askPrice','askSize','bidPrice','bidSize','mid','vwap','spread','v','ret','sigma']].dropna()
    df['Lvwap']=np.log(df.vwap)
    df['Lmid']=np.log(df.mid)
    df['LDvwap']=df.Lvwap-df.Lvwap.shift(60)
    df['LDmid']=df.Lmid-df.Lmid.shift(60)
    df=df.dropna()
    return df   

###Model is already saved from "/Dropbox/DataScience/ARIMA_model_saving.ipynb". Here loaded and added to "df_ml"
def ARIMA_(data):
    ### load model
    data=data.dropna()
    predictions_mid=ARIMA_mid(data.LDmid)
    predictions_vwap=ARIMA_vwap(data.LDvwap) 
    vwap_arima=np.exp(predictions_vwap+data.Lvwap.shift(60))
    mid_arima=np.exp(predictions_mid+data.Lmid.shift(60))
    df_ml['arima']=data.mid+vwap_arima-mid_arima
    
def ARIMA_mid(data):
    ### load model
    mid_arima_loaded = ARIMAResults.load('mid_arima.pkl')
    predictions_mid = mid_arima_loaded.predict()
    return predictions_mid

def ARIMA_vwap(data):
    ### load model
    vwap_arima_loaded = ARIMAResults.load('vwap_arima.pkl')
    predictions_vwap = vwap_arima_loaded.predict()
    return predictions_vwap

#### KALMAN moving average

##KF moving average
#https://github.com/pykalman/pykalman

# Import a Kalman filter and other useful libraries
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import poly1d

def kalman_ma(data):
    #x=data.mid
    x=data.mid
    # Construct a Kalman filter
    kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = 248,
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)

    # Use the observed values of the price to get a rolling mean
    state_means, _ = kf.filter(x.values)
    state_means = pd.Series(state_means.flatten(), index=x.index)
    df_ml['km']=state_means

### Linear Regression, sklearn, svm:SVR,linear_model
import pickle
#from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


## loading model saved from /Dropbox/DataScience/REG_model_saving.ipynb
filename_rgr = 'rgr.sav'
filename_svr = 'svr.sav'
# load the model from disk
loaded_rgr_model = pickle.load(open(filename_rgr, 'rb'))
loaded_svr_model = pickle.load(open(filename_svr, 'rb'))

def strat_lr(data,df):
    df=df.dropna()
    data=data.dropna()
    X=df[['askPrice','askSize','bidPrice','bidSize','vwap','spread','v','return','sigma']]
    y=df.mid
    predict_regr=loaded_rgr_model.predict(X)
    predict_svr=loaded_svr_model.predict(X)
    df['predict_regr']=predict_regr
    df['predict_svr']=predict_svr
    df_ml['REG']=de_normalise(data.mid,df.predict_regr)
    df_ml['SVR']=de_normalise(data.mid,df.predict_svr)
    
#### loading classification model from /Dropbox/DataScience/ML_20Sep
filename_svm_model_up = 'svm_model_up.sav'
filename_lm_model_up = 'lm_model_up.sav'
filename_svm_model_dn = 'svm_model_dn.sav'
filename_lm_model_dn = 'lm_model_dn.sav'
# load the model from disk
loaded_svm_up_model = pickle.load(open(filename_svm_model_up, 'rb'))
loaded_lm_up_model = pickle.load(open(filename_lm_model_up, 'rb'))
loaded_svm_dn_model = pickle.load(open(filename_svm_model_dn, 'rb'))
loaded_lm_dn_model = pickle.load(open(filename_lm_model_dn, 'rb'))

def classification_up_dn(data):
    X=data[['askPrice','askSize','bidPrice','bidSize','vwap','spread','v','return','sigma']]
    y1=data.U
    y2=data.D
    
    
    predict_svm_up=loaded_svm_up_model.predict(X)
    predict_lm_up=loaded_lm_up_model.predict(X)
    predict_svm_dn=loaded_svm_dn_model.predict(X)
    predict_lm_dn=loaded_lm_dn_model.predict(X)
    
    data['predict_svm_up']=predict_svm_up
    data['predict_lm_up']=predict_lm_up
    data['predict_svm_dn']=predict_svm_dn
    data['predict_lm_dn']=predict_lm_dn
    
    data['predict_svm']=data.predict_svm_up+data.predict_svm_dn
    data['predict_lm']=data.predict_lm_up+data.predict_lm_dn
    
    data['UD']=np.where(np.logical_and(data.predict_svm>0,data.predict_lm>0),1,np.where(np.logical_and(data.predict_svm<0,data.predict_lm<0),-1,0))  
       
    df_ml['UD']=data.UD

### LSTM

#df.loc[:, cols].prod(axis=1)
def lstm_processing(df):
    df=df.dropna()
    df_price=df[['mid','vwap','arima','km','REG','SVR']]
    #normalization
    dfn=normalise(df_price,12)
    dfn['UD']=df.UD
    return dfn


import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import load_model
model = load_model('21sep.h5')

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        b = dataset[i:(i+look_back), 1]
        c = dataset[i:(i+look_back), 2]
        d = dataset[i:(i+look_back), 3]
        e=  dataset[i:(i+look_back), 4]
        f = dataset[i:(i+look_back), 5]
        g=  dataset[i:(i+look_back), 6]
        dataX.append(np.c_[b,c,d,e,f,g])
        #dataX.append(b)
        #dataX.append(c)
        #dataX.append(d)
        #dataX.append(e)
        #dataX.concatenate((a,bT,cT,dT,eT),axis=1)
        dataY.append(dataset[i + look_back,0])
    return np.array(dataX), np.array(dataY)


def strat_LSTM(df_ml):
    
    #normalization
    df_lstm=lstm_processing(df_ml)
    df_lstm=df_lstm.dropna()
    dataset=df_lstm.values
    dataset = dataset.astype('float32')
    # reshape into X=t and Y=t+1
    look_back = 3
    X_,Y_ = create_dataset(dataset,look_back)
    
    # reshape input to be [samples, time steps, features]
    X_ = numpy.reshape(X_, (X_.shape[0],X_.shape[1],X_.shape[2]))
    # make predictions
    predict = model.predict(X_)
    df_lstm=df_lstm.tail(len(predict))
    df_lstm['LSTM']=predict

    #LSTM=(df_lstm.LSTM*(df_ml.mid.rolling(60).max()-df_ml.midClose.rolling(60).min()))+df_LSTM.Close.rolling(60).min()
    LSTM=de_normalise(df_ml.mid,df_lstm.LSTM,window_length=12)
    df_lstm['pred']=LSTM
    df_lstm=df_lstm.dropna()
    df_lstm=df_lstm.tail(len(df_ml))
    df_ml['LSTM']=df_lstm.pred

In [18]:
## warm up upto preprocessing
window=100
for _ in range(window):
#while True:
    iterations += 1
    string = socket_sub.recv_string()
    sym, bidPrice,bidSize,askPrice,askSize,mid,vwap,spread,v,ret,sigma = string.split()
    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))
    dt = datetime.datetime.now()
    df = df.append(pd.DataFrame({'Stock':sym,'bidPrice': float(bidPrice),'bidSize': float(bidSize),\
                                 'askPrice': float(askPrice),'askSize': float(askSize),'mid':float(mid),\
                                 'vwap':float(vwap),'spread':float(spread),'v':float(v),'ret':float(ret),\
                                 'sigma':float(sigma)},index=[dt]))
    dfn=normalise(df)
    df_arima=arima_processing(df)
    #df_ml['mid']=df.mid
    #df_ml['vwap']=df.vwap

      
    
    #print(df.tail(1))
    #print(data.tail(1))
    print(dfn.tail(1))
    #print(df_arima.tail(1))

    #x = data.to_string(header=False,index=False,index_names=False).split('\n')
    #socket_pub.send_string(x[-1])
    #print(x[-1])

AttributeError: 'str' object has no attribute 'log'

In [18]:
df_ml

,mid,vwap
2017-09-22 19:04:29.648355,249.035,249.032151
2017-09-22 19:04:29.659637,249.035,249.039524
2017-09-22 19:04:29.671420,249.035,249.039875
2017-09-22 19:04:29.682631,249.035,249.039518
2017-09-22 19:04:29.695266,249.035,249.039803
2017-09-22 19:04:29.707821,249.035,249.039614
2017-09-22 19:04:29.720365,249.035,249.038000
2017-09-22 19:04:29.732948,249.035,249.039574
2017-09-22 19:04:29.744999,249.030,249.039149
2017-09-22 19:04:29.756707,249.030,249.036071


In [27]:
## storing ML output
df_ml=pd.DataFrame()
window=100
for _ in range(window):
#while True:
    iterations += 1
    string = socket_sub.recv_string()
    sym, bidPrice,bidSize,askPrice,askSize = string.split()
    #print('%s %s %s %s %s' % (sym, bidPrice,bidSize,askPrice,askSize))
    dt = datetime.datetime.now()
    df = df.append(pd.DataFrame({'Stock':sym,'bidPrice': float(bidPrice),'bidSize': float(bidSize),'askPrice': float(askPrice),'askSize': float(askSize)},index=[dt]))
    data=preprocessing(df)
    window_length=60
    mn=data.rolling(60).min()
    mx=data.rolling(60).max()
    #data.rolling(60).max()-data.rolling(60).min()
    #dfn=(data-mn)/(mx-mn)
    #dfn=normalise(data)
    #df_arima=arima_processing(data)
    ### prediction for last 60 points
    #data=data.dropna().tail(100)
    #dfn=dfn.dropna().tail(100)
    #df_arima=df_arima.dropna().tail(100)

    #df_ml['mid']=data.mid
    #df_ml['vwap']=data.vwap
    
    #x = data.to_string(header=False,index=False,index_names=False).split('\n')
    #print(df.tail(1))
    #print(df_ml.tail())
    #print(data.tail(1))
    #socket_pub.send_string(x[-1])
    print(m) 


                           Stock  askPrice  askSize  bidPrice  bidSize  \
2017-09-22 01:17:08.799561   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.803262   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.806882   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.810625   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.814192   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.817912   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.822554   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.826793   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.830971   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.834411   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.837713   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.841132   SPY       NaN      NaN       NaN      NaN   
2017-09-22 01:17:08.844421   SPY      